In [13]:
import os
import csv
import json
from dataclasses import dataclass
from typing import List, Tuple
from PIL import Image
from tqdm import tqdm

@dataclass
class ImageLabel:
    dataset: str
    sourceImage: str
    
    runwayLabel: List[Tuple[int, int]]

"""
    Reads a CSV containing LARD data with columns:
        image (filepath relative to csv_file),
        x_A, y_A, x_B, y_B, x_C, y_C, x_D, y_D

    Produces cropped images and associated label files in 'dest_folder',
    each sized exactly H×H, where H is the original image's height.

    Cropping logic:
      - The entire vertical range (0..H) is used (no vertical trimming).
      - We find a horizontal slice of width H that is (as much as possible)
        centered on the runway. If the runway extends beyond what can fit,
        or if W < H (making an H×H square impossible), we skip.

    :param csv_file: Path to the CSV file.
    :param dest_folder: Directory where cropped images and label files will be saved.
"""
def adapt_lard(csv_file: str, dest_folder: str):
    os.makedirs(dest_folder, exist_ok=True)

    with open(csv_file, "r", newline="", encoding="utf-8") as f:
        reader = list(csv.DictReader(f, delimiter=';'))

    for row in tqdm(reader, desc="Processing rows", unit="image"):
        pts = [
            (int(row["x_A"]), int(row["y_A"])),
            (int(row["x_B"]), int(row["y_B"])),
            (int(row["x_C"]), int(row["y_C"])),
            (int(row["x_D"]), int(row["y_D"]))
        ]

        csv_dir = os.path.dirname(csv_file)
        image_path = os.path.join(csv_dir, row["image"])
        try:
            img = Image.open(image_path)
        except FileNotFoundError:
            print(f"WARNING: File not found: {image_path}")
            continue

        W, H = img.size

        if W < H:
            print(f"WARNING: Image '{image_path}' is too narrow "
                  f"({W}px) to produce a {H}x{H} square. Skipping.")
            continue

        xs = [p[0] for p in pts]
        ys = [p[1] for p in pts]
        min_x, max_x = min(xs), max(xs)
        min_y, max_y = min(ys), max(ys)

        runway_width = max_x - min_x
        if runway_width > H:
            print(f"WARNING: Runway bounding box in '{image_path}' "
                  f"width={runway_width} exceeds final crop={H}. Skipping.")
            continue

        # vertical crop is the entire height
        top = 0
        bottom = H

        # horizontal positioning for H-wide crop
        center_x = (min_x + max_x) / 2.0
        ideal_left = int(round(center_x - (H / 2)))
        left = max(0, min(ideal_left, W - H))
        right = left + H

        cropped_img = img.crop((left, top, right, bottom))
        shifted_pts = [(p[0] - left, p[1] - top) for p in pts]

        image_label = ImageLabel(
            dataset="LARD/LARD_test_real_nominal",
            sourceImage=os.path.basename(image_path),
            runwayLabel=shifted_pts
        )

        base_name = os.path.splitext(os.path.basename(image_path))[0]
        out_image_name = f"{base_name}.png"
        out_json_name = f"{base_name}.json"

        out_image_path = os.path.join(dest_folder, out_image_name)
        out_json_path = os.path.join(dest_folder, out_json_name)

        cropped_img.save(out_image_path)

        with open(out_json_path, "w", encoding="utf-8") as label_file:
            json.dump(
                {
                    "dataset": image_label.dataset,
                    "sourceImage": image_label.sourceImage,
                    "runwayLabel": image_label.runwayLabel,
                },
                label_file,
                indent=4
            )


In [14]:
adapt_lard("datasets/LARD/LARD_test_real/LARD_test_real_nominal/Test_Real_Nominal.csv", "Templates")

Processing rows:   3%|▎         | 39/1500 [00:03<02:54,  8.39image/s]

Processing rows:   3%|▎         | 51/1500 [00:05<04:02,  5.97image/s]

Processing rows:  26%|██▋       | 394/1500 [01:33<01:56,  9.46image/s]

Processing rows:  74%|███████▍  | 1116/1500 [04:47<00:34, 10.98image/s]

Processing rows: 100%|██████████| 1500/1500 [06:46<00:00,  3.69image/s]
